In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
# Install TensorFlow

import tensorflow as tf
import numpy as np
import base64
import sys
from tensorflow.python.client import device_lib

print("Python:"+sys.version)
print("TF:"+tf.__version__)
print("GPU:"+str(tf.test.is_gpu_available())+" CUDA:"+str(tf.test.is_built_with_cuda()))
#print(device_lib.list_local_devices())

Python:3.5.6 |Anaconda, Inc.| (default, Aug 26 2018, 16:05:27) [MSC v.1900 64 bit (AMD64)]
TF:2.0.0
GPU:True CUDA:True


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28),name='Flatten'),
  #tf.keras.layers.Dense(128, activation='relu',name='Dense1'),
  tf.keras.layers.Dense(128,name='Dense1'),
  #tf.keras.layers.LeakyReLU(alpha=0.2),    
  tf.keras.layers.ReLU(),    
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(10, activation='softmax',name='Output')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [4]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 46us/sample - loss: 0.3198 - accuracy: 0.9061
Epoch 2/5
60000/60000 [==============================] - 2s 38us/sample - loss: 0.1641 - accuracy: 0.9508
Epoch 3/5
60000/60000 [==============================] - 2s 39us/sample - loss: 0.1262 - accuracy: 0.9610
Epoch 4/5
60000/60000 [==============================] - 2s 39us/sample - loss: 0.1062 - accuracy: 0.9675
Epoch 5/5
60000/60000 [==============================] - 2s 38us/sample - loss: 0.0920 - accuracy: 0.9717


In [5]:
model.evaluate(x_train,  y_train, verbose=1,batch_size=128)

60000/1 [=======] - 1s 9us/sample - loss: 0.1025 - accuracy: 0.9867


[0.044796775067349276, 0.9867]

In [6]:
def SaveModel(my_model,fileSTR):
    totalbytes=0
    data=[]
    Wmodel = open("./Mokka/"+fileSTR+".weights", "wb")
    for x in my_model.weights:
        nn = x.numpy()
        T = nn
        #if (nn.ndim == 4):
        #    T = np.transpose(nn, (3,0,1,2))
        #elif (nn.ndim == 3):
        #    T = np.transpose(nn, (2,0,1))
        #else:
        #    T = nn.transpose()
        v = np.ndarray.tobytes(T)
        Wmodel.write(bytearray(v))
        totalbytes+=len(v)
        print(x.name, len(v)," dims:",nn.ndim," ", T.shape)
        #if "bias" in x.name:
         #   print(T)
        data.append(base64.b64encode(v).decode("utf-8"))
    Wmodel.close()
    print("Total bytes:"+str(totalbytes))

In [7]:
SaveModel(model,"DENSE")

Dense1/kernel:0 401408  dims: 2   (784, 128)
Dense1/bias:0 512  dims: 1   (128,)
Output/kernel:0 5120  dims: 2   (128, 10)
Output/bias:0 40  dims: 1   (10,)
Total bytes:407080


In [8]:
model.save('simple.h5')

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
Dense1 (Dense)               (None, 128)               100480    
_________________________________________________________________
re_lu (ReLU)                 (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
Output (Dense)               (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.get_layer('Output').weights

[<tf.Variable 'Output/kernel:0' shape=(128, 10) dtype=float32, numpy=
 array([[ 0.25838962,  0.06922607, -0.24579443, ...,  0.30715936,
         -0.31119227, -0.32856742],
        [ 0.30996573,  0.09394977, -0.57722765, ..., -0.23561454,
          0.08649647, -0.44042537],
        [ 0.2241466 , -0.45880908, -0.20618825, ...,  0.18304335,
          0.02498914,  0.10425311],
        ...,
        [ 0.01515807, -0.4769282 ,  0.11009397, ...,  0.25769907,
         -0.15365885, -0.49681112],
        [ 0.11247961, -0.5195554 ,  0.12638982, ..., -0.20718886,
          0.15486637,  0.16420089],
        [-0.4565279 , -0.17802586, -0.20560831, ..., -0.1250861 ,
          0.2110254 ,  0.19146816]], dtype=float32)>,
 <tf.Variable 'Output/bias:0' shape=(10,) dtype=float32, numpy=
 array([-0.00675962, -0.05633172, -0.13950533, -0.20233499, -0.00163202,
        -0.03614089, -0.0379517 ,  0.05244585,  0.24413656,  0.12814283],
       dtype=float32)>]